In [1]:
import os

In [2]:
%pwd

'c:\\Users\\meabh\\VS Code Projects\\Kidney-Tumor-classification-DL-using-MLFlow-CICD\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\meabh\\VS Code Projects\\Kidney-Tumor-classification-DL-using-MLFlow-CICD'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


## this code is an Entity

In [6]:
### Whatever defined in the "Config.yaml" file , all need to return in the above code.

In [7]:
from cnnClassifier.constants import *       # In the constants folder, there root file path is given
from cnnClassifier.utils.common import read_yaml, create_directories   # In the utils folder, we have imported required function


## This is configuration ralated things

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])     # It will create a folder

    def get_data_ingestion_config(self) -> DataIngestionConfig:   # This function will return the value which is defined in the Class "DataIngestionConfig"
        config = self.config.data_ingestion      # using "data_ingestion" we are extracting the things which is given by "data_ingestion" as per "config.yaml" file

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
## Now prepare the components

In [10]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


# This cell and below cell is the Components

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:   ## This function is used to download the data
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL   # extracting the URL from configuration
            zip_download_dir = self.config.local_data_file    # preparing download link
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):     # This function is used to Extact Zip file
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
## update the Pipeline

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-18 09:47:46,173: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-18 09:47:46,178: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-18 09:47:46,180: INFO: common: created directory at: artifacts]
[2023-11-18 09:47:46,181: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-18 09:47:46,183: INFO: 4196335540: Downloading data from https://drive.google.com/file/d/1JdOFjlz4nPTDpSV9EZJ_kW_cAZWc65fP/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1JdOFjlz4nPTDpSV9EZJ_kW_cAZWc65fP
From (redirected): https://drive.google.com/uc?/export=download&id=1JdOFjlz4nPTDpSV9EZJ_kW_cAZWc65fP&confirm=t&uuid=5037f813-7544-435c-bf6e-f56a379dad22
To: c:\Users\meabh\VS Code Projects\Kidney-Tumor-classification-DL-using-MLFlow-CICD\artifacts\data_ingestion\data.zip
100%|██████████| 134M/134M [00:22<00:00, 6.07MB/s] 

[2023-11-18 09:48:11,423: INFO: 4196335540: Downloaded data from https://drive.google.com/file/d/1JdOFjlz4nPTDpSV9EZJ_kW_cAZWc65fP/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [ ]:
## If all the above works Properly , Then convert it into a Modular Coding